<a href="https://colab.research.google.com/github/ashwinlagji/neural-network/blob/main/DL_Assignment_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DL Assignemnt CIFAR 10 using DL

### Question No.1.
Vision Dataset: CIFAR-10- It dataset consists of 60000 32x32 colour images in 10 classes.
Please find your dataset from the link- https://www.tensorflow.org/datasets/catalog/cifar10. (5 marks)
Prepare a python notebook (recommended- use Google Colab) to build, train and evaluate a deep neural network on
the CIFAR-10 dataset. Read the instructions carefully.

*** 

## 1. Import Libraries/Dataset 
-  Import required libraries (recommended- use tensorflow/keras library).
-  Import the dataset (use Google Drive if required).
-  Check the GPU available (recommended- use free GPU provided by Google Colab).

In [ ]:
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers

from matplotlib import pyplot as plt


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(tf.test.gpu_device_name());
print(device_lib.list_local_devices());

print('Xtrain \n', x_train[10,10])
print('Xtest \n', x_test[10,10])
print('Ytrain \n', y_train[10,])
print('Ytest \n', y_test[10,])

*** 

## 2. Data Visualization 


- Plot at least one sample from each class of the dataset (use matplotlib/seaborn/any other library).
- Print the shapes of train and test data.

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'];
fig, axes = plt.subplots(ncols=7, nrows=3, figsize=(17, 8))
index = 0
for i in range(3):
    for j in range(7):
        axes[i,j].set_title(labels[y_train[index][0]])
        axes[i,j].imshow(x_train[index])
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        index += 1
plt.show()

In [ ]:
print('Shape of x_train is {}'.format(x_train.shape))
print('Shape of x_test is {}'.format(x_test.shape))
print('Shape of y_train is {}'.format(y_train.shape))
print('Shape of y_test is {}'.format(y_test.shape))

*** 

## 3. Data Pre-processing 
- Bring the train and test data in the required format.

In [ ]:
# Normalizing
x_train = x_train.reshape((50000, 32*32*3)) 
x_train = x_train.astype('float32') / 255

x_test = x_test.reshape((10000, 32*32*3)) 
x_test = x_test.astype('float32') / 255

#One hot encoding
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

print(x_train[0])
print(y_test_cat[0])

*** 

## 4. Model Building
- Sequential Model layers- Use AT LEAST 3 dense layers with appropriate input for each. Choose the best number for hidden units and give reasons.
- Add L2 regularization to all the layers.
- Add one layer of dropout at the appropriate position and give reasons.
- Choose the appropriate activation function for all the layers.
- Print the model summary.

In [ ]:
# Create a model object

dnnModel = models.Sequential()

# Layer 1 = input layer
# specify the input size in the first layer.

dnnModel.add(layers.Dense(500, activation='relu', input_shape= (32*32*3,)))

# Layer 2 = hidden layer 
dnnModel.add(layers.Dense(500, kernel_regularizer=regularizers.l2(0.0001) ,activation='relu'))

# Layer 3 = hidden layer 
dnnModel.add(layers.Dense(500, kernel_regularizer=regularizers.l2(0.0001), activation='relu'))

# Layer 4 = hidden layer 
dnnModel.add(layers.Dense(500, kernel_regularizer=regularizers.l2(0.0001),  activation='relu'))

# Layer 5 = hidden layer 
dnnModel.add(layers.Dense(500, kernel_regularizer=regularizers.l2(0.0001),  activation='relu'))

# Layer 6 = hidden layer 
dnnModel.add(layers.Dense(500, kernel_regularizer=regularizers.l2(0.0001),  activation='relu'))

# Layer 7 = output layer
dnnModel.add(layers.Dense(10, activation='softmax'))

dnnModel.summary()

***

## 5. Model Compilation

- Compile the model with the appropriate loss function.
- Use an appropriate optimizer. Give reasons for the choice of learning rate and its value.
- Use accuracy as metric.

In [ ]:
#
dnnModel.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'] )

***

### 6. Model Training 
- Train the model for an appropriate number of epochs (print the train and validation accuracy/loss for
each epoch). Use the appropriate batch size.
- Plot the loss and accuracy history graphs. Print the total time taken for training.

In [ ]:
h  = dnnModel.fit( x_train , y_train_cat , epochs=50, batch_size=64)

***
## 7. Model Evaluation
- Print the final test/validation loss and accuracy.
- Print confusion matrix and classification report for the validation dataset. Write a summary for the best and worst performing class and the overall trend.

In [ ]:
#
#
#
#
print('Final training loss \t', h.history['loss'][-1])
print('Final training accuracy ', h.history['accuracy'][-1])


In [ ]:
# testing the model

testLoss, testAccuracy = dnnModel.evaluate( x_test, y_test_cat)

In [ ]:
print('Testing loss \t', testLoss)
print('Testing accuracy ', testAccuracy)